<a href="https://colab.research.google.com/github/pietrolira/PPGEEC2318-Redes-Neurais-e-Deep-Learning/blob/main/Lista02/Q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Questão 4

**a-)** Utilizando uma rede NARX no caso uma rede neural perceptron de múltiplas camadas
com realimentação global para fazer a predição de um passo, isto é $\hat{x}(n+1)$, da série temporal.

**Resposta:**
Utilizando uma rede NARX (rede neural perceptron de múltiplas camadas com realimentação global), foi feita a predição de um passo da série temporal $\hat{x}(n+1)$. A rede foi treinada com os dados da série $x(n)$ gerados pela equação:

$
x(n) = v(n) + \beta v(n-1)v(n-2), \quad \beta = 0{,}5
$

com $v(n) \sim \mathcal{N}(0, 1)$.

**b-)** Repetir o problema utilizando uma rede LSTM.
Avalie o desempenho das redes recorrentes mostrando a curva da série temporal, a curva
de predição e a curva do erro de predição $e(n+1) = x(n+1) - \hat{x}(n+1)$.

**Resposta:**
O mesmo problema foi repetido utilizando uma rede LSTM e as curvas obtidas foram:

* **Curva da série temporal:** sequência real de $x(n)$.
* **Curva de predição:** saída da rede $\hat{x}(n+1)$.
* **Curva do erro de predição:** $e(n+1) = x(n+1) - \hat{x}(n+1)$.

---

Gerar a série temporal x(n)

x(n)=v(n)+βv(n−1)v(n−2)
com
𝛽
=
0.5
β=0.5, onde
𝑣
(
𝑛
)
v(n) é ruído branco gaussiano (média 0, variância 1).

In [ ]:
import numpy as np
N = 1000
beta = 0.5
np.random.seed(42)
v = np.random.randn(N)
x = np.zeros(N)
for n in range(2, N):
    x[n] = v[n] + beta * v[n-1] * v[n-2]

Preparar os dados

a) Para NARX:
Usamos os valores passados como entradas:
𝑥
(
𝑛
−
1
)
,
𝑥
(
𝑛
−
2
)
x(n−1),x(n−2) → prever
𝑥
(
𝑛
+
1
)
x(n+1).

In [ ]:
def preparar_dados_narx(x, atrasos=2):
    X, Y = [], []
    for i in range(atrasos, len(x)-1):
        X.append([x[i-1], x[i-2]])
        Y.append(x[i+1])
    return np.array(X), np.array(Y)

b) Para LSTM:
Mesmo conjunto, mas com shape 3D: [samples, timesteps, features].

In [ ]:
def preparar_dados_lstm(x, atrasos=2):
    X, Y = [], []
    for i in range(atrasos, len(x)-1):
        X.append([[x[i-2]], [x[i-1]]])
        Y.append(x[i+1])
    return np.array(X), np.array(Y)

Dividir treino/teste e normalizar os dados
python
Copiar
Editar


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Preparar os dados
X_narx, y_narx = preparar_dados_narx(x)
X_lstm, y_lstm = preparar_dados_lstm(x)

# Divisão
X_train_narx, X_test_narx, y_train_narx, y_test_narx = train_test_split(X_narx, y_narx, test_size=0.2, shuffle=False)
X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(X_lstm, y_lstm, test_size=0.2, shuffle=False)

# Normalização
scaler = MinMaxScaler()
X_train_narx = scaler.fit_transform(X_train_narx)
X_test_narx = scaler.transform(X_test_narx)

X_train_lstm = scaler.fit_transform(X_train_lstm.reshape(-1, 2)).reshape(-1, 2, 1)
X_test_lstm = scaler.transform(X_test_lstm.reshape(-1, 2)).reshape(-1, 2, 1)

Treinar e testar os modelos

a) NARX (simulada com MLP)

LSTM
python
Copiar
Editar


In [ ]:
from tensorflow.keras.layers import LSTM

model_lstm = Sequential([
    LSTM(10, input_shape=(2, 1)),
    Dense(1)
])
model_lstm.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
model_lstm.fit(X_train_lstm, y_train_lstm, epochs=50)

Avaliação e gráfico
python
Copiar
Editar

In [ ]:
import matplotlib.pyplot as plt

# Predição
y_pred_narx = model_narx.predict(X_test_narx).flatten()
y_pred_lstm = model_lstm.predict(X_test_lstm).flatten()

erro_narx = y_test_narx - y_pred_narx
erro_lstm = y_test_lstm - y_pred_lstm

# Plot
plt.figure(figsize=(14, 10))
plt.subplot(3, 1, 1)
plt.plot(y_test_narx, label='Real')
plt.plot(y_pred_narx, label='Predição NARX')
plt.title('Predição da Série Temporal - Rede NARX')
plt.legend()

plt.subplot(3, 1, 2)
plt.plot(y_test_lstm, label='Real')
plt.plot(y_pred_lstm, label='Predição LSTM')
plt.title('Predição da Série Temporal - Rede LSTM')
plt.legend()

plt.subplot(3, 1, 3)
plt.plot(erro_narx, label='Erro NARX')
plt.plot(erro_lstm, label='Erro LSTM')
plt.title('Erro de Predição')
plt.legend()

plt.tight_layout()
plt.show()